In [1]:
import numpy as np
import cv2
import argparse
import glob
import os

In [2]:
SAVE_PATH = r'D:\dataset\DATA_Python\endoscope\ROI_data\\'
images_PATH = r'D:\dataset\DATA_Python\endoscope\cap_endoscope_frame_selected'

images = glob.glob(images_PATH + '/*.jpg')

green_color = (0, 255, 0)
white_color = (225, 225, 225)
refpt = []
cropping = False

In [3]:
#클릭한 좌표 저장 및 클릭한 위치 표시
def click_and_crop(event, x, y, flags, param):
    global refpt, cropping
    refpt_tmp = []
    
    if event == cv2.EVENT_LBUTTONDOWN:
        cropping = True

    elif event == cv2.EVENT_LBUTTONUP:
        refpt_tmp.append(x)
        refpt_tmp.append(y)
        
        refpt.append(refpt_tmp)
        cropping = False
        
        print(refpt)
        print(refpt_tmp)

        cv2.rectangle(image, (x, y), (x+1, y+1),
                     (255, 0, 0), 2)
        cv2.imshow("image", image)

In [4]:
#ROI poly 이미지를 보여줌
def Roi_poly(coordinates):
    pt1 = np.array(coordinates, np.int32)
    img = cv2.polylines(image, [pt1], True, green_color, 2)
    
    cv2.imshow("polylines", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
def ROI_SAVE(coordinates, SAVE_PATH, image, basename):

    #좌표를 받아서 배열
    pt2 = np.array(coordinates, np.int32)
    
    #img의 shape를 확인하여 마스크img 생성 준비
    height, width, channel = image.shape
    back_img = np.zeros((height, width), np.uint8)
    
    #ROI 색칠 추출
    back_img = cv2.fillConvexPoly(back_img, pt2, white_color)


    #ROI save
    cv2.imwrite(SAVE_PATH + '{}'.format(basename[:-4]) +'_mask' + '.jpg', back_img) 

In [7]:
#roi를 추출할 사진
for i in range(len(images)):
    image = cv2.imread(images[i])
    basename = os.path.basename(images[i])
    clone = image.copy()
    cv2.namedWindow('image')


    #ROI 추출 
    while True:
        cv2.imshow("image", image)
        key = cv2.waitKey(1) & 0xFF
        cv2.setMouseCallback('image', click_and_crop)
    
        #키보드 r을 누르면 기존에 클릭한 점을 초기화시킨다
        if key == ord("r"):
            image = clone.copy()
            global refpt
            refpt = []
    
        #키보드 c를 누르면 ROI를 구해줌
        if key == ord("c"):
            Roi_poly(refpt)
        
        if key == ord("s"):
            ROI_SAVE(refpt, SAVE_PATH, image, basename)
        
        #키보드 q를 누르면 다음 img or 나감   
        elif key == ord("q"):
            refpt = []
            
            break

    cv2.destroyAllWindows()